In [1]:
#import des librairies
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# On reshape les images et on definit les paths
IMAGE_SIZE = [224, 224]
train_path = 'cell_images/Train'
valid_path = 'cell_images/Test'

In [3]:
# On initialise le modele avec les poids imagenet
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 11s 0us/step


In [4]:
# On n'entraine pas les couches existantes
for layer in vgg.layers:
  layer.trainable = False

 # pour obtenir le nb de classes
folders = glob('cell_images/Train/*')
  

# les couches
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

# On cree le model
model = Model(inputs=vgg.input, outputs=prediction)

# Structure du model
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
# Choix de la fction cout et optimizer
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


# On importe les images du dataset

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('cell_images/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('cell_images/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# On fit le model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Found 22038 images belonging to 2 classes.
Found 5520 images belonging to 2 classes.
Epoch 1/10
689/689 [==============================] - 385s 559ms/step - loss: 0.3686 - accuracy: 0.8420 - val_loss: 0.9197 - val_accuracy: 0.8283
Epoch 2/10
689/689 [==============================] - 380s 551ms/step - loss: 0.2800 - accuracy: 0.8899 - val_loss: 0.2469 - val_accuracy: 0.9096
Epoch 3/10
689/689 [==============================] - 365s 529ms/step - loss: 0.2788 - accuracy: 0.8935 - val_loss: 0.2019 - val_accuracy: 0.8815
Epoch 4/10
689/689 [==============================] - 364s 529ms/step - loss: 0.2537 - accuracy: 0.9057 - val_loss: 0.2540 - val_accuracy: 0.9172
Epoch 5/10
689/689 [==============================] - 364s 529ms/step - loss: 0.2516 - accuracy: 0.9079 - val_loss: 0.5534 - val_accuracy: 0.8966
Epoch 6/10
689/689 [==============================] - 365s 530ms/step - loss: 0.2480 - accuracy: 0.9094 - val_loss: 0.5734 - val_accuracy: 0.8841
Epoch 7/10
689/689 [===================